In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet')
test = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet')

In [ ]:
def compute_feats(df):
    """Generate features dynamically from text columns."""
    for col in ["response_a", "response_b", "prompt"]:
        # Text length features
        df[f"{col}_len"] = df[f"{col}"].str.len()

        # Character counting features
        df[f"{col}_spaces"] = df[f"{col}"].str.count("\s")
        df[f"{col}_punct"] = df[f"{col}"].str.count(",|\.|!")
        df[f"{col}_question_mark"] = df[f"{col}"].str.count("\?")
        df[f"{col}_special_chars"] = df[f"{col}"].str.count("\W")
        df[f"{col}_digits"] = df[f"{col}"].str.count("\d")
        df[f"{col}_letters"] = df[f"{col}"].str.count("[a-zA-Z]")
        df[f"{col}_json"] = df[f"{col}"].str.lower().str.count("json")

    # Presence of prompt words in responses
    df["response_a_prompt_overlap"] = df.apply(
        lambda x: len(set(x["prompt"].split()).intersection(set(x["response_a"].split()))) / len(set(x["prompt"].split())) if len(set(x["prompt"].split())) > 0 else 0, axis=1
    )
    df["response_b_prompt_overlap"] = df.apply(
        lambda x: len(set(x["prompt"].split()).intersection(set(x["response_b"].split()))) / len(set(x["prompt"].split())) if len(set(x["prompt"].split())) > 0 else 0, axis=1
    )
    return df

In [ ]:
# Apply feature engineering
train = compute_feats(train)
test = compute_feats(test)

In [ ]:
# Define feature columns
X = train.drop(["winner", "id"], axis=1)
y = train["winner"]
categorical_features = ["prompt", "response_a", "response_b"]

feats = [col for col in X.columns if col in test.columns]
X = X[feats]
test = test[feats]

# Define weighted importance for selected features
selected_features = [col for col in X.columns if "_len" in col or "_special_chars" in col or "_prompt_overlap" in col]
feature_weights = {col: 2.0 if col in selected_features else 1.0 for col in X.columns}

# Train the CatBoost model
# KFold Cross-Validation and Parameter Tuning
kf = KFold(n_splits=10, shuffle=True, random_state=42)

catboost_params = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [200, 500],
    'l2_leaf_reg': [1, 3, 5],
}

catboost_param_grid = ParameterGrid(catboost_params)
best_catboost_score = 0
best_catboost_params = {}

for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    for params in catboost_param_grid:
        model = CatBoostClassifier(
            **params, 
            cat_features=categorical_features, 
            random_state=42, 
            task_type='GPU',
            feature_weights=feature_weights
        )
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=0)
        val_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, val_pred)
        
        if accuracy > best_catboost_score:
            best_catboost_score = accuracy
            best_catboost_params = params
print("Best CatBoost Params:", best_catboost_params)
print("Best CatBoost Accuracy:", best_catboost_score)

In [ ]:
final_model = CatBoostClassifier(
    **best_catboost_params, 
    cat_features=categorical_features, 
    random_state=42, 
    task_type='GPU'
)
final_model.fit(X, y, verbose=100)

# Predictions on the test set
test['winner'] = final_model.predict(test)

In [ ]:
submission = pd.read_csv('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/sample_submission.csv')
submission['winner'] = test['winner']
submission.to_csv('/kaggle/working/submission.csv', index=False)